In [1]:
from torchview import draw_graph
import torch
from transformers import MistralForSequenceClassification, LlamaTokenizerFast, AutoModel, AutoModelForSequenceClassification, AutoTokenizer

import graphviz
graphviz.set_jupyter_format('png')

'svg'

In [2]:
hf_repo = 'h2oai/h2o-danube2-1.8b-base'
device = 'cpu'

In [3]:
model_name = hf_repo.split('/')[1]

In [4]:

model_normal = AutoModel.from_pretrained(
    hf_repo, 
    num_labels=3, 
    # device_map=device,
    # torch_dtype=torch.bfloat16
)

model_sequence_classifier = AutoModelForSequenceClassification.from_pretrained(
    hf_repo, 
    num_labels=3, 
    # device_map=device,
    # torch_dtype=torch.bfloat16
)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at h2oai/h2o-danube2-1.8b-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

tokenizer = AutoTokenizer.from_pretrained(hf_repo)

In [6]:

# if tokenizer.pad_token is None:
#     num_added_toks = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
#     assert num_added_toks == 1
#     model.resize_token_embeddings(len(tokenizer))
    # model.config.pad_token_id = tokenizer.pad_token_id


In [7]:

premises = [
    "and",
]

hypotheses = [
    'my'
]

if tokenizer.pad_token is None:
    num_added_tokens = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
    assert num_added_tokens == 1
    for model in [model_normal, model_sequence_classifier]:
        model.resize_token_embeddings(len(tokenizer))
        # model.config.pad_token = tokenizer.pad_token
        model.config.pad_token_id = tokenizer.pad_token_id


inputs = tokenizer(premises, hypotheses, truncation='only_first', padding="max_length",
                    max_length=min(tokenizer.model_max_length, 8192), add_special_tokens=True, return_tensors='pt')

inputs

{'input_ids': tensor([[32000, 32000, 32000,  ..., 32000,   304,   586]]), 'attention_mask': tensor([[0, 0, 0,  ..., 0, 1, 1]])}

In [8]:


normal_model_graph = draw_graph(model_normal, input_data=inputs, save_graph=True, filename=f"graphed_{model_name}_normal.svg", directory=r'/mnt/c/Users/lachl/PycharmProjects/bart-large-mnli-recreation/exploration/notebooks/images')

KeyboardInterrupt: 

In [17]:


classifier_model_graph = draw_graph(model_sequence_classifier, input_data=inputs, save_graph=True, filename=f"graphed_{model_name}_classifier.svg", directory=r'/mnt/c/Users/lachl/PycharmProjects/bart-large-mnli-recreation/exploration/notebooks/images')


In [ ]:
# type(model_graph.visual_graph)

In [ ]:

# fie_ext = 'png'
# temp_img = 'temp_file'
# my_graph = graphviz.Source(model_graph)
# my_graph.render(temp_img,format=fie_ext, view=False)

In [ ]:
# model_graph.visual_graph